In [4]:
import glob
import json

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("yappy_data_w_clip_embeds.csv")
print(df.shape)
df.head(2)

(107225, 4)


,link,description,file_key,downloaded
0,https://cdn-st.rutubelist.ru/media/e2/97/f9164...,NaN,f9164f8a41479f961d64842154a7,True
1,https://cdn-st.rutubelist.ru/media/0f/48/8a1ff...,#diy #постановка #юмор #комедия,8a1ff7324073947a31e80f71d001,True


In [3]:
df.isnull().sum(0)

link               0
description    14741
file_key           0
downloaded         0
dtype: int64

In [7]:
npy_files = sorted(
    glob.glob("/home/letfd/lct2024_data/data/clip_numpy_embeds/*npy"),
    key=lambda name: int(name.split('_')[-1].replace('.npy', ''))
)

In [15]:
merged_embeds = np.concatenate([np.load(fp) for fp in npy_files])

In [18]:
assert merged_embeds.shape[0] == df.shape[0]

np.save("/home/letfd/lct2024_data/data/merged_clip_embeds.npy", merged_embeds, allow_pickle=False)

In [21]:
merged_embeds.shape

(107225, 512)

In [19]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [22]:
client.recreate_collection(
    collection_name="lct_clip",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

/tmp/ipykernel_2209/2659649393.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [23]:
%%time

pl = iter([row.to_dict() for _, row in df.iterrows()])

CPU times: user 6.89 s, sys: 42 ms, total: 6.93 s
Wall time: 6.93 s


In [25]:
%%time

client.upload_collection(
    collection_name="lct_clip",
    vectors=merged_embeds,
    payload=pl,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=512,  # How many vectors will be uploaded in a single request?
)

CPU times: user 19.8 s, sys: 3.56 s, total: 23.4 s
Wall time: 34.1 s


In [31]:
from qdrant_client import QdrantClient

class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        # initialize Qdrant client
        self.qdrant_client = QdrantClient("http://localhost:6333")
        
    def search(self, text: str):
        # Convert text query into vector
        # vector = self.model.encode(text).tolist()
        text_inp = self.processor(
            text = [text], images=None, return_tensors="pt"
        )
        text_emb = self.model.get_text_features(**text_inp).detach().numpy()[0]

        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=text_emb,
            query_filter=None,  # If you don't want any filters for now
            limit=10,  # 5 the most closest results is enough
        )
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

In [32]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
neural_searcher = NeuralSearcher(collection_name="lct_clip")


In [58]:
%%time

neural_searcher.search(text='mountain climbing')

CPU times: user 99.8 ms, sys: 6.84 ms, total: 107 ms
Wall time: 37.7 ms


[{'description': None,
  'downloaded': True,
  'file_key': 'ee7641da421e848a137d7bef892d',
  'link': 'https://cdn-st.rutubelist.ru/media/b6/c8/ee7641da421e848a137d7bef892d/fhd.mp4'},
 {'description': 'I think this must be my favourite view number one  Do you agree This was a hike to Store Trolltind a couple of weeks ago. 1788 meters above sea level.',
  'downloaded': True,
  'file_key': 'd6ae183d433093f0ccb23eb3ace9',
  'link': 'https://cdn-st.rutubelist.ru/media/74/3d/d6ae183d433093f0ccb23eb3ace9/fhd.mp4'},
 {'description': 'Пик Бзерпи 2482 м\n\nНевероятная локация с панорамным видом на горы на все 360 градусов.',
  'downloaded': True,
  'file_key': '36a9f07848f3a0e2492cb470ed19',
  'link': 'https://cdn-st.rutubelist.ru/media/14/71/36a9f07848f3a0e2492cb470ed19/fhd.mp4'},
 {'description': '#наспорте #тренировка #упражнения',
  'downloaded': True,
  'file_key': '9def330b4bd4924fbacc75753968',
  'link': 'https://cdn-st.rutubelist.ru/media/e3/b2/9def330b4bd4924fbacc75753968/fhd.mp4'},
 {'

In [56]:
%%time

import requests
resp = requests.get("http://127.0.0.1:8000/search", params={'query': 'home pets'})

CPU times: user 6.03 ms, sys: 408 µs, total: 6.44 ms
Wall time: 38.3 ms


In [57]:
resp.json()

[{'description': '#doglife #puppylife #puppygram #puppies #dogvideos #puppyoftheday #dogoftheday #dachshund',
  'downloaded': True,
  'file_key': '48d47d84418382e9d8601bbc71fa',
  'link': 'https://cdn-st.rutubelist.ru/media/86/21/48d47d84418382e9d8601bbc71fa/fhd.mp4'},
 {'description': 'Emergency Family Meeting 🚨🙈 I think it went quite well… #whocanrelate \nFeaturing my cousins @tomthechi',
  'downloaded': True,
  'file_key': '820037c240d2ad384e56316ec1b2',
  'link': 'https://cdn-st.rutubelist.ru/media/9a/fc/820037c240d2ad384e56316ec1b2/fhd.mp4'},
 {'description': '#наука #научное #научпоп #учимсявместе #самообучение #репетитор #образование',
  'downloaded': True,
  'file_key': 'fd58952d4f0081f8031d12a3a147',
  'link': 'https://cdn-st.rutubelist.ru/media/75/10/fd58952d4f0081f8031d12a3a147/fhd.mp4'},
 {'description': '#путешествия #journey #туризм #природа #животные #саванна #красивыеместа',
  'downloaded': True,
  'file_key': 'f89d65b74531aab80a63daaf1d74',
  'link': 'https://cdn-st.ru